In [ ]:
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
#import librosa.display
#mne의 time_frequency 또는 librosa는 오류가 있어 scipy를 사용함. 200203
from scipy import signal
#import random
import mne
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import scipy.stats as ss
from scipy import signal
MITmon_matrix7=[-1,'T7-FT9',-1,-1,'FP1-F7','F7-T7','T7-P7','P7-O1','FP1-F3','F3-C3','C3-P3','P3-O1',-1,'FZ-CZ','CZ-PZ',-1,
                'FP2-F4', 'F4-C4','C4-P4','P4-O2','FP2-F8','F8-T8','T8-P8','P8-O2', -1,'FT10-T8', -1, -1]
## FT9, FT10 체널 빠진 부분 0으로 대체. 13번 환자에서 특히 많았음.

channel_order_list=['T7-FT9','FP1-F7','F7-T7','T7-P7','P7-O1','FP1-F3','F3-C3','C3-P3','P3-O1','FZ-CZ','CZ-PZ',
                'FP2-F4', 'F4-C4','C4-P4','P4-O2','FP2-F8','F8-T8','T8-P8-0','P8-O2', 'FT10-T8']#T8-P8이 2개가 있어, MNE에서 불러올때 -0 -1 꼬리표 붙음

Mon_matrix=MITmon_matrix7
Mon_matrix_col_no=7 # 7 또는 5 중 7으로 진행.
Mon_matrix_row_no=4 #row는 4로 고정

sampling_rate=200 #MIT는 256, SNUH는 200
freq_seg=0.5 #freq domain 기본 간격
freq_total_n=sampling_rate/freq_seg # freq domain 간격 총 개수
freq_upper_range=40 #freq upper range Hz
freq_n=int(freq_upper_range/freq_seg) #일단 현재는 40
sec_frame=20 #LSTM의 frame 길이
edf_list = os.listdir('edf_file/')
make_ch_list=['FP1-F7','F7-T3','T3-T1','T3-T5','T5-O1','FP1-F3','F3-C3','FZ-CZ','C3-P3','P3-O1','FP2-F4','F4-C4','CZ-PZ','C4-P4','P4-O2',
'FP2-F8','F8-T4','T4-T2','T4-T6','T6-O2']

In [ ]:
#edf_list = os.listdir('edf_file/')
for order in range(0,len(edf_list)):
    freq_dict = {}
    sampling_rate = 200
    raw = mne.io.read_raw_edf('edf_file' +'/'+ edf_list[order], preload=True)
    raw.filter(l_freq=0.5,h_freq=30)
    channel_list=raw.ch_names   
    data=raw._data
    channel_name_list = []
    for ch in range(len(channel_list)):
        channel_name_list.append([channel_list[ch].split('-')[0].upper(),ch])
    for ma in range(len(make_ch_list)):
        odd_list = []
        for fr in range(len(channel_name_list)):
            for be in range(len(channel_name_list)):
                if channel_name_list[fr][0]+str('-')+ channel_name_list[be][0] == make_ch_list[ma]:
                    odd_list.append([fr,be])
                    #print(ma)
                    make_data=data[fr] - data[be]
                    f, t, Zxx = signal.stft(make_data,fs=sampling_rate,nperseg=400,noverlap=200,nfft=freq_total_n)
                    freq_dict[make_ch_list[ma]] = {}
                    Zxx=Zxx.astype('float')
                    freq_dict[make_ch_list[ma]]=Zxx[0:int(freq_n),:-1]  
                    break
    with open('/SNU_data/pickle_file/stft_filter/'+edf_list[order].split('.')[0]+'.pickle', 'wb') as f:
        pickle.dump(freq_dict, f, pickle.HIGHEST_PROTOCOL) 


In [ ]:
for i in range(len(make_ch_list)):
    make_ch_list[i]=make_ch_list[i].upper()

In [ ]:

odd_list = []
for ma in range(len(make_ch_list)):
    for fr in range(len(channel_name_list[:-1])):
        for be in range(len(channel_name_list[:-1])):
        
            if channel_name_list[fr][0]+str('-')+ channel_name_list[be][0] == make_ch_list[ma]:
                odd_list.append([fr,be])
                print(make_ch_list[ma])
                break

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel('SNUH_VEM_200326.xlsx')

In [ ]:
time_dict = {}
for d in range(len(df)):
    time_dict[d+1] = {}
    s = str(df.iloc[d,:][9]).replace('-','/')
    start_time = datetime.strptime(s, "%Y/%m/%d %H:%M:%S")      
    e = str(df.iloc[d,:][10]).replace('-','/')
    end_time = datetime.strptime(e, "%Y/%m/%d %H:%M:%S")  
    tmin=end_time-start_time 
    tmin=int(tmin.total_seconds())
    time_dict[d+1]['start'] = s
    time_dict[d+1]['end'] = e
    time_dict[d+1]['time_second'] = tmin

In [ ]:
with open('pickle_file/time_dict.pickle', 'wb') as f:
    pickle.dump(time_dict, f)

In [ ]:
sz_dict = {}
for d in range(len(df)):
    sz_dict[d+1] = {}
    if df.iloc[d,:][7] != 0 :
        sz_dict[d+1]['num'] = df.iloc[d,:][7]
        for sz in range(int(df.iloc[d,:][7])):
            sz_dict[d+1][sz+1] = {}
            sz_s =str(df.iloc[d,:][16+(2*sz)]).replace('-','/')
            start_time = datetime.strptime(sz_s, "%Y/%m/%d %H:%M:%S")      
            sz_e =str(df.iloc[d,:][16+(2*sz)+1]).replace('-','/')
            end_time = datetime.strptime(sz_e, "%Y/%m/%d %H:%M:%S")  
            tmin=end_time-start_time 
            tmin=int(tmin.total_seconds())
            sz_dict[d+1][sz+1]['start'] = sz_s
            sz_dict[d+1][sz+1]['end'] = sz_e
            sz_dict[d+1][sz+1]['time_second'] = tmin

In [ ]:
dis_dict = {}
for d in range(len(df)):
    dis_dict[d+1] = {}
    if str(df.iloc[d,:][12]) != 'NaT':    
        if str(df.iloc[d,:][14]) != 'NaT':
            dis_dict[d+1]['num'] = 2
            for dis in range(2):
                dis_dict[d+1][dis+1] = {}
                dis_s =str(df.iloc[d,:][12+(dis*2)]).replace('-','/')
                start_time = datetime.strptime(dis_s, "%Y/%m/%d %H:%M:%S")      
                dis_e =str(df.iloc[d,:][12+(dis*2)+1]).replace('-','/')
                end_time = datetime.strptime(dis_e, "%Y/%m/%d %H:%M:%S")  
                tmin=end_time-start_time 
                tmin=int(tmin.total_seconds())
                dis_dict[d+1][dis+1]['start'] = dis_s
                dis_dict[d+1][dis+1]['end'] = dis_e
                dis_dict[d+1][dis+1]['time_second'] = tmin
        else:
            dis_dict[d+1]['num'] = 1
            for dis in range(1):
                dis_dict[d+1][dis+1] = {}
                dis_s =str(df.iloc[d,:][12+(dis)]).replace('-','/')
                start_time = datetime.strptime(dis_s, "%Y/%m/%d %H:%M:%S")      
                dis_e =str(df.iloc[d,:][12+(dis)+1]).replace('-','/')
                end_time = datetime.strptime(dis_e, "%Y/%m/%d %H:%M:%S")  
                tmin= end_time-start_time 
                tmin=int(tmin.total_seconds())
                dis_dict[d+1][dis+1]['start'] = dis_s
                dis_dict[d+1][dis+1]['end'] = dis_e
                dis_dict[d+1][dis+1]['time_second'] = tmin

In [ ]:
with open('pickle_file/dis_dict.pickle', 'wb') as f:
    pickle.dump(dis_dict, f)

In [ ]:
reoff_dict = {}
for d in range(len(df)):
    reoff_dict[d+1] = {}
    if str(df.iloc[d,:][11]) != 'NaT':     
        #reoff_dict[d+1]
        reoff_s =str(df.iloc[d,:][11]).replace('-','/')
        reoff_time = datetime.strptime(reoff_s, "%Y/%m/%d %H:%M:%S")      
        start =str(df.iloc[d,:][9]).replace('-','/')
        start_time = datetime.strptime(start, "%Y/%m/%d %H:%M:%S")  
        tmin=reoff_time-start_time 
        tmin=int(tmin.total_seconds())
        reoff_dict[d+1]['start'] = reoff_s        
        reoff_dict[d+1]['time_second'] = tmin


In [ ]:
with open('pickle_file/reoff_dict.pickle', 'wb') as f:
    pickle.dump(reoff_dict, f)